In [15]:
import re

In [16]:
from opentrons.simulate import simulate, format_runlog

In [17]:
# read the file
# protocol_file = open('../protocols/station-C-qpcr-map.py')
protocol_file = open('../experiments/20200328/station-C-qpcr-map.py')
# simulate() the protocol, keeping the runlog
runlog, _bundle = simulate(protocol_file, custom_labware_paths=['../labware/'])

In [18]:
gottip = False
source_wells = set()
dest_plates = dict()

for action in runlog:
    text = action['payload']['text'].strip()
    if text.startswith("Picking up tip"):
        gottip = True
    
    if text.startswith("Aspirating"):
        well, plate = re.match(r'Aspirating .* from ([A-Z][0-9]+) of (.* on .*) at .*', text).groups()
        source_wells.add((plate, well))
        
    if text.startswith("Dispensing"):
        dest_well, dest_plate = re.match(r'Dispensing .* into ([A-Z][0-9]+) of (.* on .*) at .*', text).groups()
        if dest_plate not in dest_plates:
            dest_plates[dest_plate] = dict()
            
        if dest_well not in dest_plates[dest_plate]:
            dest_plates[dest_plate][dest_well] = set()
        
        for source_well in source_wells:
            dest_plates[dest_plate][dest_well].add(source_well)
        
    if text.startswith("Dropping tip"):
        gottip = False
        source_wells.clear()
        

In [23]:
REAGENT_LOCATIONS = {
    'Reaction': 'A1',
    'Endogenous': 'B1',
    'Standard': 'B2',
    'Water': 'B3',
    'Negative': 'B3', # Synonym for water
    'IEC RNA': 'C1',
    'PCD 8': 'C5',
    'PCD 7': 'C6',
    'PCD 6': 'D1',
    'PCD 5': 'D2',
    'PCD 4': 'D3',
    'PCD 3': 'D4',
    'PCD 2': 'D5',
    'PCD 1': 'D6'
}

reagent_names = dict()
for name, loc in REAGENT_LOCATIONS.items():
    reagent_names[loc] = name

for name, plate in dest_plates.items():
    print(name)
    for well_name, well in sorted(plate.items(), key=lambda item: (int(item[0][1:]), item[0][0])):
        print(well_name)
        for source_plate, source_well in well:
            if source_plate == 'Applied Biosystems 96 Aluminum Block 200 µL on Temperature Module on 4':
                continue
                
            if source_plate == 'Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5':
                print("  " + reagent_names[source_well])
            else:
                print("  " + source_well)
        print()
    print()
    print()

Applied Biosystems 96 Aluminum Block 200 µL on Temperature Module on 4
A1
  A1
  Reaction

B1
  B1
  Reaction

C1
  Reaction
  C1

D1
  Reaction
  D1

E1
  Reaction
  E1

F1
  Reaction
  F1

G1
  G1
  Reaction

H1
  H1
  Reaction

A2
  A2
  Reaction

B2
  B2
  Reaction

C2
  Reaction
  C2

D2
  Reaction
  D2

E2
  E2
  Reaction

F2
  Reaction
  F2

G2
  G2
  Reaction

H2
  H2
  Reaction

A3
  Reaction
  A3

B3
  Reaction
  B3

C3
  Reaction
  C3

D3
  D3
  Reaction

E3
  E3
  Reaction

F3
  Reaction
  F3

G3
  G3
  Reaction

H3
  Reaction
  H3

A4
  A4
  Reaction

B4
  Reaction
  B4

C4
  Reaction
  C4

D4
  D4
  Reaction

E4
  Reaction
  E4

F4
  F4
  Reaction

G4
  G4
  Reaction

H4
  H4
  Reaction

A5
  A5
  Reaction

B5
  Reaction
  B5

C5
  C5
  Reaction

D5
  Reaction
  D5

E5
  Reaction
  E5

F5
  Reaction
  F5

G5
  Reaction
  G5

H5
  Reaction
  H5

A6
  A6
  Reaction

B6
  B6
  Reaction

C6
  C6
  Reaction

D6
  Reaction
  D6

E6
  Reaction
  E6

F6
  Reaction
  F6

G6
  Reac

In [17]:
MASTER_MIX_MAP = '''
Standard	Standard	Standard
Standard	Standard	Standard
Standard	Standard	Standard
				Standard	Standard	Standard
				Standard	Standard	Standard
				Standard	Standard	Standard		Standard	Standard	Standard
								Standard	Standard	Standard
								Standard	Standard	Standard
'''

In [34]:
master_mix_labels = [""] * 12*8
for i, row in enumerate(MASTER_MIX_MAP.strip('\n ').split('\n')):
    for j, label in enumerate(row.split('\t')):
        master_mix_labels[i*12+j] = label

In [35]:
master_mix_labels

['Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '']

In [19]:
master_mix_labels = re.split(r'[\n\t]', MASTER_MIX_MAP.strip('\n '))

In [22]:
MASTER_MIX_MAP.strip('\n ')

'Standard\tStandard\tStandard\nStandard\tStandard\tStandard\nStandard\tStandard\tStandard\n\t\t\t\tStandard\tStandard\tStandard\n\t\t\t\tStandard\tStandard\tStandard\n\t\t\t\tStandard\tStandard\tStandard\t\tStandard\tStandard\tStandard\n\t\t\t\t\t\t\t\tStandard\tStandard\tStandard\n\t\t\t\t\t\t\t\tStandard\tStandard\tStandard'

In [20]:
master_mix_labels

['Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Standard',
 'Standard',
 'Standard']